This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.7) and depends on Magma (tested using version
2.27-1).

In this notebook, we find generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1 with $\#C(\mathbb{F}_2) = 5$.  Allow 2 hours for completion.

In [1]:
load("preamble.sage")

Read from disk the precomputed data about codimension-4 flats in $\mathbf{P}^9$.

In [2]:
F = GF(2)

In [3]:
with open("genus6-flats.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
    l = [[vector(F, v) for v in vecs] for vecs in l]
len(l)

57

Construct the points of the Grassmannian $X$ in $\mathbf{P}^9$, then verify the Plucker relations.

In [4]:
coords = {}
for V in VectorSpace(F, 5).subspaces(2):
    M = V.matrix()
    M.set_immutable()
    coords[M] = vector(M.minors(2))
S = list(coords.keys())
len(S)

155

In [5]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = (x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23)

In [6]:
assert all(gen(*coords[M]) == 0 for M in coords for gen in quads)

Generate all quadratic monomials, then evaluate these on the points of $X$.

In [7]:
monos2 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

55

In [8]:
coords2 = {}
V0 = VectorSpace(F, 10)
for x in V0:
    x.set_immutable()
    coords2[x] = vector(F, (mu(*x) for mu in monos2))

For each flat in the list, iterate over 5-tuples of rational points on the intersection of the flat with the Grassmannian and solve for quadrics passing through exactly those points.

In [9]:
def vecs_to_gens(vecs):
    return tuple(sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs)

In [10]:
def redundancy(gens, F=F, P=P, monos2=monos2, quads=quads):
    return [vector(F, ((gen*y).coefficient(mu) for mu in monos2)) for gen in gens for y in P.gens()] + \
       [vector(F, (gen.coefficient(mu) for mu in monos2)) for gen in quads]

In [11]:
curves = defaultdict(list)
W = VectorSpace(F, 55)
for vecs in l:
    V1 = Matrix(vecs).row_space()
    V = Matrix(vecs).right_kernel()
    gens = vecs_to_gens(V1.basis())
    pts = [coords[x] for x in S if coords[x] in V]
    print(len(pts), [(s, len(curves[s])) for s in curves])
    for v in pts:
        v.set_immutable()
    W1 = W.quotient(redundancy(gens))
    perp = Matrix([coords2[x] for x in pts])
    for s in [5]:
        for pts1 in itertools.combinations(pts, s):
            target = vector(F, (0 if x in set(pts1) else 1 for x in pts))
            for w in solve_right_iterator(perp, target, redundancy, gens):
                gens1 = sum(w[i]*monos2[i] for i in range(55))
                curves[(s,)].append(gens + (gens1,))

9 []
9 [((5,), 16128)]
11 [((5,), 32256)]
7 [((5,), 47040)]
9 [((5,), 57792)]
7 [((5,), 73920)]
11 [((5,), 84672)]
15 [((5,), 99456)]
7 [((5,), 105336)]
5 [((5,), 116088)]
13 [((5,), 118136)]
13 [((5,), 128432)]
13 [((5,), 138728)]
9 [((5,), 149144)]
5 [((5,), 165272)]
7 [((5,), 167320)]
11 [((5,), 178072)]
11 [((5,), 193304)]
15 [((5,), 208536)]
9 [((5,), 214500)]
13 [((5,), 230628)]
17 [((5,), 240924)]
7 [((5,), 243990)]
7 [((5,), 254742)]
19 [((5,), 265494)]
11 [((5,), 266958)]
15 [((5,), 281742)]
11 [((5,), 287748)]
11 [((5,), 302532)]
15 [((5,), 317316)]
15 [((5,), 323156)]
7 [((5,), 329036)]
9 [((5,), 339788)]
13 [((5,), 355916)]
9 [((5,), 366332)]
11 [((5,), 380668)]
11 [((5,), 395452)]
19 [((5,), 410236)]
11 [((5,), 411640)]
23 [((5,), 427768)]
19 [((5,), 428284)]
11 [((5,), 429772)]
15 [((5,), 445004)]
11 [((5,), 450968)]
27 [((5,), 467096)]
23 [((5,), 467386)]
11 [((5,), 467626)]
11 [((5,), 482858)]
15 [((5,), 497642)]
15 [((5,), 503648)]
7 [((5,), 514400)]
19 [((5,), 535904)

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2$ using point counting and for $i=3,4$ using commutative algebra.

In [12]:
F4 = GF(4)
S4 = []
for V in VectorSpace(F4, 5).subspaces(2):
    M = V.matrix()
    v = vector(M.minors(2))
    i = min(j for j in range(10) if v[j])
    assert v[i] == 1
    v.set_immutable()
    S4.append(v)
len(S4)

5797

In [13]:
tmp = [s for s in curves if len(s) == 1]
tmp2 = set(t[:2] for t in targets6)
for (s1,) in tmp:
    gens1 = 0
    for gens in curves[(s1,)]:
        if gens1 != gens[:-1]:
            gens1 = gens[:-1]
            pts2 = [x for x in S4 if all(gen(*x) == 0 for gen in gens1)]
        s2 = sum(1 for x in pts2 if gens[-1](*x) == 0)
        if (s1, s2) in tmp2:
            curves[(s1,s2)].append(gens)
    del curves[(s1,)]

In [14]:
[(s, len(curves[s])) for s in curves]

[((5, 13), 59679), ((5, 11), 106208), ((5, 15), 49168)]

In [15]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + quads + tuple(y^(2^n) + y for y in P.gens()))
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [16]:
for n in range(3, 5):
    tmp = set(t[:n] for t in targets6)
    tmp2 = list(curves.keys())    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

[((5, 13, 14), 7465), ((5, 11, 11), 19721), ((5, 15, 5), 1915)]
[((5, 13, 14, 25), 1271), ((5, 11, 11, 31), 9830), ((5, 11, 11, 39), 300), ((5, 15, 5, 35), 15)]


Close out this case.

In [17]:
proj = magma.ProjectiveSpace(P)
curves = {s: [gens + quads for gens in curves[s]] for s in curves}
closeout(curves, X=proj, genus=6)

Number of curves found: 281
Number of isomorphism classes found: 31
Number of covers found: 1
(5, 15, 5, 35, 20, 45) $.1^2*$.1^6 + ($.1^3 + $.1^2 + 1)*$.1^5 + ($.1^3 + $.1^2 + $.1 + 1)*$.1^4 + ($.1^5 + $.1^4 + $.1^3 + $.1)*$.1^3 + ($.1^5 + $.1^4 + $.1^3 + $.1)*$.1^2 + ($.1^7 + $.1^6 + $.1 + 1)*$.1 + $.1^8 + $.1^7 + $.1^4 + $.1^3 + $.1^2 + 1
All covers recorded!
Total time: 136 minutes (8203 seconds)
